In [1]:
import websocket
import mysql.connector
import json
import ast

Connect to Database

In [17]:
def insert_table(table_name, D):
    try:
        print(D)
        mydb = mysql.connector.connect(
          host="localhost",
          user="root",
          password="password",
          database="crypto"
        )
        cursor = mydb.cursor()
        query = f"insert into {table_name} values({D['E']},{D['k']['t']},{D['k']['T']},{D['k']['o']},{D['k']['c']},{D['k']['h']},{D['k']['l']}, {D['k']['v']},{D['k']['n']},{D['k']['x']},{D['k']['q']},{D['k']['V']},{D['k']['Q']})"
        cursor.execute(query)
        mydb.commit()
    except:
        print("There is a problem with MySQL")   
        
def get_raw_table(symbol,interval):
    try:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="password",
            database="crypto"
        )
        cursor = mydb.cursor(dictionary=True)
        query = f"SELECT m01_tab_s FROM m01_mapping WHERE m01_symbol = '{symbol}' AND m01_interval = '{interval}';"
        cursor.execute(query)
        result = cursor.fetchone()
        if result:
            return result['m01_tab_s']
        else:
            return None
    except Exception as e:
        print(f"There is a problem with MySQL table name: {e}")
        return None
        
def get_last_timestamp(symbol, interval):
    try:
        table_name = get_raw_table(symbol,interval)
        print(table_name)
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="password",
            database="crypto"
        )
        cursor = mydb.cursor(dictionary=True)
        query = f"SELECT MAX(event_time) as event_time FROM {table_name};"
        cursor.execute(query)
        result = cursor.fetchone()
        if result and result['event_time']:
            return result['event_time']
        else:
            return None
    except Exception as e:
        print(f"There is a problem with MySQL last event time: {e}")
        return None
    

Stream from Binance API

In [23]:
api_key = 'HGuZ0EOwG2CgAAq7CLx7JjLCTNHd2NL5BzqBAfa8qiuQkcVBaUySB'
api_secret = 'QGaXTPSjsROXj24O7y5AxYH1CjAWNFqssmmHOEb5HCAzoIfY3gEs3ZhSGgfDg2C6'

def on_message(ws, message):
    try:
        data = json.loads(message)
        insert_table('s01_btcusdt_15m',data)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")    

def on_error(ws, error):
    print(error)
    
def on_close(close_message):
    print(close_message)
    
def streamKline(symbol, interval):
    last_time_stamp = get_last_timestamp(symbol,interval)
    socket = f'wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}'
    ws = websocket.WebSocketApp(socket,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.run_forever()

In [24]:
streamKline('btcusdt', '15m')

s01_btcusdt_15m
Handshake status 400 Bad Request


In [22]:
get_last_timestamp('btcusdt', '15m')

s01_btcusdt_15m


1706854381827